In [4]:
import pandas as pd
import os
from pathlib import Path
pd.set_option('display.max_colwidth', 100)

df = pd.DataFrame(None, columns = ['id', 'mask', 'gender', 'age',  'totalLabel', 'customLabel', 'useMask', 'isCorrect', 'actualAge', 'ageModTen', 'errorM', 'errorG', 'errorA', 'error', 'isAug', 'path'])

img_dir = '/opt/ml/input/data/train/images_face_crop'
out_path = '/opt/ml/input/data/train/new_test.csv'

train_df =  pd.read_csv('/opt/ml/input/data/train/original_train.csv')

def age_group(x):
    if x < 30:
        return 0
    elif x < 60:
        return 1
    else:
        return 2

for index, line in enumerate(train_df.iloc):
    for file in list(os.listdir(os.path.join(img_dir, line['path']))):
        if file[0] == '.':
            continue
        if file.split('.')[0] == 'normal':
            mask = 2
            isCorrect = -1
        elif file.split('.')[0] == 'incorrect_mask':
            mask = 1
            isCorrect = 0
        else:
            mask = 0
            isCorrect = 1
            
        gender = 0 if line['gender'] == 'male' else 1
        age = age_group(line['age'])
        
        useMask = 1 if mask != 2 else 0
        actualAge = line['age']
        ageModTen = actualAge // 10
        totalLabel = mask * 6 + gender * 3 + age
        customLabel = mask * 12 + gender * 6 + ageModTen
        
        data = {
            'id': line['id'],
            'mask': mask,
            'gender': gender,
            'age': age,
            'useMask': useMask,
            'isCorrect': isCorrect,
            'actualAge': actualAge,
            'ageModTen' : ageModTen,
            'errorM': 0,
            'errorG': 0,
            'errorA': 0,
            'error': 0,
            'totalLabel': totalLabel,
            'customLabel': customLabel,
            'isAug': 0,
            'path': os.path.join(img_dir, line['path'], file),
        }
        df = df.append(data, ignore_index=True)

df.to_csv(out_path, index=False)